# Desafio PLN e Chatbots - SERPRO

**Página do Desafio:** [https://www.kaggle.com/c/desafio-ia-2020-pln-chatbots/overview](https://www.kaggle.com/c/desafio-ia-2020-pln-chatbots/overview)

---
### Skynet

Robson de Sousa Martins<br>
[https://www.robsonmartins.com](https://www.robsonmartins.com)

___

Este desafio foi solucionado com a implementação da classificação das frases (perguntas de chatbot) em três etapas:

1. **Pré-classificação das frases por palavras-chave**: Nesta etapa, as frases são classificadas através de comparação de suas palavras com um banco de palavras-chave prestabelecido. Assim, já são identificadas frases de assuntos não-relacionados aos grupos de bots sugeridos (e classificadas como `0: Nenhum`), e frases de assuntos amplamente conhecidos, como por exemplo as que contém a palavra `COVID-19` (classe `2: Covid`).

2. **Classificação das frases restantes através de algoritmo**: Nessa fase, um algoritmo de classificação é selecionado, e realiza a classificação das frases restantes, baseado no aprendizado realizado com a massa de dados de treino.

3. **Remoção de classificação de baixa probabilidade**: Frases classificadas com probabilidade mais baixa que um número mínimo definido, são desclassificadas, ou seja, clasificadas para `0: Nenhum`. Isso elimina alguns dos equívocos produzidos pelo algoritmo classificador e aumenta a qualidade do resultado.


É importante que o conjunto de palavras-chave seja bem escolhido para que reflita fortemente cada uma das classes/assuntos propostos.

Para este desafio, o classificador por palavras-chave foi implementado de maneira simples, sem realizar cálculo probabilístico da frequência ou peso de palavras-chave (ou seja, se ele simplesmente encontrar a palavra-chave na primeira classe avaliada, atribui uma probabilidade de 1.0 ou 100%, sem avaliar as outras classes subsequentes).

Numa aplicação real, o conjunto de palavras-chave poderia ser dinâmico, atualizado periodicamente, alimentado por um banco de dados de palavras mais citadas por cada assunto, tal como numa nuvem de palavras, ou *trend topics* oriundos de um *data mining* de redes sociais, por exemplo.

# Bibliotecas utilizadas

In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import string

import nltk

from nltk.corpus import stopwords as sw
from nltk.tokenize import RegexpTokenizer

from unicodedata import normalize
from datetime import datetime, timedelta

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from statistics import mean

In [ ]:
# Vetorizador
from sklearn.feature_extraction.text import TfidfVectorizer

# Clssificadores
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier

## Download de Módulos

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Desliga warnings desnecessários
pd.set_option('mode.chained_assignment',None)

# Classes

### DummyVectorizer

Vetorizador "dummy": não realiza nenhuma transformação de dados. Mantém apenas a interface de um vetorizador básico, padrão do sklearn.

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator

# Vetorizador "dummy": não realiza nenhuma transformação de dados
# Mantém apenas a interface de um Vetorizador básico

class DummyVectorizer(BaseEstimator):
    
    def __init__(self):
        pass
    
    def fit(self, raw_documents, y=None):
        return self
    
    def transform(self, raw_documents):
        return np.array(raw_documents).reshape(-1, 1)
    
    def fit_transform(self,raw_documents, y=None):
        self.fit(raw_documents, y)
        return self.transform(raw_documents)

### KeywordClassifier

Um Classificador por palavras-chave (keywords). Este classificador foi construido com a mesma interface de um classificador do sklearn.

O método fit() recebe um array de palavras-chave e classes a que se referem, em ordem de prioridade.
O método predict() classifica frases de acordo com as palavras-chave treinadas com fit().
Usa um stemmer de língua portuguesa para corresponder palavras similares (com mesma raiz).

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from nltk.tokenize import RegexpTokenizer
from nltk.stem import RSLPStemmer
from unicodedata import normalize

# Classificador por palavras-chave (keywords).
# Este classificador foi construido com a mesma interface de um classificador do sklearn.
# O método fit() recebe um array de palavras-chave e classes a que se referem, em ordem de prioridade.
# Se uma classe não contém palavras-chave (array vazio), será usada como padrão para registros que não forem classificados.
# O método predict() classifica frases de acordo com as palavras-chave treinadas com fit().
# Usa um stemmer de língua portuguesa para corresponder palavras similares (com mesma raiz).
# Parâmetros:
# verbose (default = True): indica se deve exibir informações detalhadas do progresso da classificação.

class KeywordClassifier(BaseEstimator, ClassifierMixin):
    
    verbose = True # verboso por padrao
    classes_, X_, y_ = [], [], [] # classes, X e y

    _cache = [] # classes e palavras-chave aprendidas com fit()
    _y_default = None # classe default, caso não saiba classificar
    
    _tokenizer = RegexpTokenizer(r'\w+') # tokenizador
    _stemmer = RSLPStemmer() # stemmer de lingua portuguesa
    
    # construtor
    def __init__(self,verbose=True):
        self.verbose = verbose
        if self.verbose:
            print('KeywordClassifier: verbose mode started.')
    
    # fit(X,y): preenche _cache com as classes/keywords
    def fit(self, X, y, sample_weight=None):
        # check that X and y have correct shape
        X, y = check_X_y(X, y, dtype=['object','int64','float64'])
        # store the classes seen during fit
        self.classes_ = unique_labels(y) # classes ordenadas
        self.X_ = X # palavras-chave
        self.y_ = y # classes
        self._cache = []
        for i in range(len(y)):
            y_data = y[i]
            X_data = X[i][0]
            if len(X_data) == 0: self._y_default = y_data # pega classe default
            self._cache.append({'y':y_data,'X':X_data}) # memoriza keywords/classes em _cache
            if self.verbose:
                print(f'fitting keywords {i+1}/{len(y)} ...')
        return self

    # predict(X): retorna classes (y) baseado na matriz de probabilidades
    # retornada por predict_proba(X)
    def predict(self, X):
        # calcula matriz de probabilidades 
        _proba = self.predict_proba(X)
        _predict = []
        # varre cada sample da matriz e retorna classe correspondente para cada sample
        for sample in _proba:
            _cidx = -1
            _cmax = 0.0
            for i in range(len(sample)):
                if sample[i] > _cmax:
                    _cmax = sample[i]
                    _cidx = i
            _predict.append(self.classes_[_cidx]) # retorna classe (a de maior probabilidade na matriz)
        return _predict
    
    # predict_proba(X): retorna matriz de probabilidades de cada sample, para
    # cada classe inferida
    # 0.0 = não há correspondência nos keywords (0% de probabilidade de ser da classe)
    # 1.0 = há correspondência nos keywords (100% de probabilidade de ser da classe)
    def predict_proba(self, X):
        # check is fit had been called
        check_is_fitted(self)
        # input validation
        X = check_array(X,dtype=['object','int64','float64'])
        _proba = []
        _count = []
        _total = 0
        # inicializa contador de samples/classe
        for j in range(len(self.classes_)):
            _count.append(0)
        # varre cada sample de entrada
        for i in range(len(X)):
            phrase = X[i][0]  # uma frase de entrada
            # inicializa matriz de probabilidades do sample
            _proba_sample = []
            for j in range(len(self.classes_)):
                _proba_sample.append(0.0)
            # procura correspondência de keywords
            y = self._search_keywords(phrase)
            if y == None: y = self._y_default  # se não encontrou, retorna classe default
            # varre classes ordenadas
            for j in range(len(self.classes_)):
                if y == self.classes_[j]: # se a classe coincide
                    _proba_sample[j] = 1.0  # registra na matriz a probabilidade 1.0 (100%)
                    _count[j] = _count[j] + 1  # incrementa contagem de samples
                    if y != self._y_default: _total = _total + 1  # incrementa total de samples classificados
                    break
            _proba.append(_proba_sample)
            if self.verbose and ((i+1) == 1 or (i+1) % 100 == 0 or (i+1) == len(X)):
                print(f'predict sample {i+1}/{len(X)} ...')
        # exibe relatório, se verboso
        if self.verbose and _total != 0:
            print(f'{_total}/{len(X)} ({round(_total*100/len(X),1)}%) samples predicted.')
            print('-----------------')
            print(' class | samples ')
            print('-----------------')
            for j in range(len(self.classes_)): 
                print(f'{self.classes_[j]:^7}|{_count[j]:^9}')
            print('-----------------')
        return _proba
    
    # predict_log_proba(X): mesmo que predict_proba(X), porém resultados em log (base e)
    def predict_log_proba(self, X):
        return np.log(self.predict_proba(X))
    
    # partial_fit(X,y): implementado como o mesmo que fit(X,y)
    def partial_fit(self, X, y, classes=None, sample_weight=None):
        return self.fit(X, y, sample_weight)

    # busca palavras no _cache de keywords, e retorna a classe (y), se encontrada correspondência,
    # ou None se não.
    def _search_keywords(self, text):
        tokens = self._tokenizer.tokenize(text.lower())
        found = []
        kwparts = []
        for i in range(len(self._cache)): # keywords classificados por prioridade
            data = self._cache[i]
            X = data['X']
            y = data['y']
            if len(X) == 0: X = [''] # sem keywords
            for kword_phrase in X:
                kword_phrase = kword_phrase.strip()
                kwparts = []
                found = []
                if kword_phrase == '': # classe default (sem keywords)
                    found.append(y)
                    kwparts.append('')
                    break
                if kword_phrase.find(' ') != -1: # operação and
                    kwparts = kword_phrase.split()
                else:
                    kwparts.append(kword_phrase)
                for kwitem in kwparts:
                    as_is = kwitem[0].isupper()
                    if as_is: # keyword como é (sem stemmer)
                        skw = kwitem.lower()
                    else:
                        skw = self._stem(kwitem) # keyword flexionada (stemmer para achar raiz da palavra)
                    # remove acentos
                    skw = self._ascii(skw)
                    for word in tokens:
                        if word == '': continue
                        # remove acentos
                        word = self._ascii(word)
                        if (as_is and word == skw) or (not as_is and word.startswith(skw)):
                            found.append(y)
                            break
                if len(kwparts) != 0 and len(found) == len(kwparts):
                    break
            if len(kwparts) != 0 and len(found) == len(kwparts):
                break
        if len(found) != 0 and len(found) == len(kwparts):
            return found[0]
        else:
            return None
    
    # retorna a raiz (flexão) das palavras
    def _stem(self, text):
        _phrase = []
        _tokens = self._tokenizer.tokenize(text)
        for word in _tokens:
            _phrase.append(self._stemmer.stem(word))
        return " ".join(_phrase)
    
    # remove acentos
    def _ascii(self, text):
        return normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')


# Funções

In [ ]:
# Realiza uma limpeza bem básica de um texto, preparando-o para classificação. 
def limpar_texto(texto):
    # Converte para minúsculas
    texto = texto.lower()
    # Remove números
    texto = re.sub(r'[0-9]+',' ',texto)
    # Remove pontuacao
    texto = texto.translate(str.maketrans(string.punctuation,' '*len(string.punctuation)))
    # Remove espacos extras
    texto = re.sub(r'\s+',' ',texto)
    # Remove stopwords
    tokens = tokenizer.tokenize(texto)
    tokens = [palavra.strip() for palavra in tokens if palavra not in stopwords]
    texto = ' '.join(tokens)  
    # Remove acentos
    texto = normalize('NFKD',texto).encode('ASCII','ignore').decode('ASCII')
    # cria dict de palavras unicas
    # remove palavras menores que 2 caracteres
    tokens = tokenizer.tokenize(texto)
    fdist = nltk.FreqDist(tokens)
    tokens = [palavra.strip() for palavra, freq in fdist.items() if len(palavra) >= 2]
    texto = ' '.join(tokens)  
    return texto

In [ ]:
# Calcula métricas de desempenho do classificador. Fique à vontade para incluir outras métricas que julgar úteis.
# Lembre-se, todavia, que o desafio utiliza a métrica F1 (macro) para avaliação dos resultados. 
# Referência: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

def get_metrics(y_test, y_predicted, average='macro'):  
    precision = round(precision_score(y_test, y_predicted, pos_label=1, average=average, zero_division=0),4)             
    recall = round(recall_score(y_test, y_predicted, pos_label=1, average=average, zero_division=0),4)
    f1 = round(f1_score(y_test, y_predicted, pos_label=1, average=average, zero_division=0),4)
    accuracy = round(accuracy_score(y_test, y_predicted),4)
    return accuracy, precision, recall, f1

In [ ]:
# Treina um classificador, otimiza hiperparâmetros,
# avalia performance e retorna métricas de desempenho
def build(X,y,vec,est,grid=None,n_splits=5,fit=True):
    y_preds = []
    scores_accuracy = []
    scores_precision = []
    scores_recall = []
    scores_f1 = []
    est_name = est.__class__.__name__
    vec_name = vec.__class__.__name__
    print(f'Testando o classificador {est_name} - {vec_name} ...')
    # massa de dados
    try:
        X_data = vec.transform(X.tolist()).toarray()
    except:
        X_data = vec.transform(X.tolist())
    y_data = y
    # divida massa em folds
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    if grid != None:
        # faz o gridsearch
        # uso F1 Macro como métrica
        clf = GridSearchCV(est,grid,scoring='f1_macro',n_jobs=-1,cv=n_splits,verbose=100)
        clf.fit(X_data,y_data)
        estimator = clf.best_estimator_
    else:
        estimator = est
        class DummyCLF: best_params_ = {}
        clf = DummyCLF() 
    # faz validacao cruzada com kfold
    for fold, (tr_idx, ts_idx) in enumerate(kf.split(X_data,y_data)):
        # separa massas de treino/teste
        X_tr, X_ts = X_data[tr_idx], X_data[ts_idx]
        y_tr, y_ts = y_data[tr_idx], y_data[ts_idx]
        if fit:
            # treina
            estimator.fit(X_tr, y_tr)
        # avalia a performance
        y_pr = estimator.predict(X_ts)
        accuracy, precision, recall, f1 = get_metrics(y_ts, y_pr)
        scores_accuracy.append(accuracy)
        scores_precision.append(precision)
        scores_recall.append(recall)
        scores_f1.append(f1)

    # obtém as métricas de desempenho - o quanto nosso classificador acertou?
    accuracy = sum(scores_accuracy) / len(scores_accuracy);
    precision = sum(scores_precision) / len(scores_precision);
    recall = sum(scores_recall) / len(scores_recall);
    f1 = sum(scores_f1) / len(scores_f1);

    return estimator, est_name, vec_name, accuracy, precision, recall, f1, clf.best_params_

# Inicialização

### Chatbots Disponívels para Treinamento

A seguir a relação dos chatbots disponibilizados para o desafio.

O "*id*"  é o identificador do chatbot. Ele está explícito aqui para evitar quaisquer dúvidas. O(s) classificador(es) deve(m) ser treinado(s) usando **ESTES** identificadores específicos. 

O identificador **0** (zero) deverá ser atribuído às perguntas que forem consideradas como **não** direcionadas a nenhum dos bots abaixo. Isso visa simular um ambiente real de orquestração, onde esse tipo de situação ocorre com frequência. Tais perguntas existirão apenas no arquivo de submissão, sem rótulos.

In [ ]:
bots = [
    {'id':0, 'nome':'Nenhum'},
    {'id':1, 'nome':'Alistamento Militar'},
    {'id':2, 'nome':'COVID'},
    {'id':3, 'nome':'Login Único'},
    {'id':4, 'nome':'IRPF - Perguntão 2020'},
    {'id':5, 'nome':'PGMEI - Programa Gerador de DAS do Microempreendedor Individual'},
    {'id':6, 'nome':'POC Selo Turismo Responsável'},
    {'id':7, 'nome':'Cadastur - Cadastro dos Prestadores de Serviços Turísticos'},
    {'id':8, 'nome':'Tuberculose'}
]

### Palavras-chave

As palavras-chave fortemente relacionadas aos assuntos (classes) são uma boa fonte de dados para uma classificação prévia.

In [ ]:
# Matriz de palavras-chave
#
# ordem: prioridade de avaliação
# quaisquer palavras dentro de 'palavras' servem para classificar
# palavras em MAIÚSCULO são avaliadas da forma como são (sem flexão)
# palavras em minúsculo são flexionadas (exemplo: 'declarar' => 'declar')
# expressões com mais de uma palavra, separadas por espaço, são avaliadas em conjunto (é necessário encontrar TODAS palavras-chave da expressão para classificar)
keywords = [
    {'id': 0, 'palavras': [ # Assuntos não-relacionados: LGPD, INSS, Eleições, Trânsito/Detran
        'LGPD',
        'PROVA VIDA',
        'TITULO eleitor','eleitor','votar','URNA','TRE','TSE',
        'DETRAN','DENATRAN','CNH','CARTEIRA MOTORISTA','CARTEIRA HABILITACAO','primeira HABILITACAO','HABILITACAO PROVISORIA','CARTEIRA PROVISORIA',
        'AUTO ESCOLA','RENAVAM','RENACH','AGENTE TRANSITO','aula pratica','dirigir','condutor'
    ]},
    {'id': 1, 'palavras': [ # Alistamento Militar
        'alistamento','militar','EXERCITO','AERONAUTICA','MARINHA','FORCAS ARMADAS','FORCA servir','FORCA aerea','reservista','CTSM','juramento BANDEIRA',
        'INCORPORACAO','CERTIFICADO DISPENSA','EXCESSO CONTINGENTE','SELECAO GERAL','CPOR','NPOR','CRDI','CERTIDAO REGISTRO DADOS INDIVIDUAIS',
        'SELECAO dispensa','documento dispensa','TIRO GUERRA','TG'
    ]},
    {'id': 6, 'palavras': [ # Selo Turismo Responsável
        'SELO turismo','SELO responsavel','SELO servico','SELO declarar','SELO higiene','SELO empreendimento','SELO atividade','SELO estabelecimento',
        'SELO cadastro','SELO EMBRATUR','SELO CADASTUR','turismo RESPONSAVEL'
    ]},
    {'id': 7, 'palavras': [ # Cadastur
        'CADASTUR','EMBRATUR','servico turismo','GUIA turismo'
    ]},
    {'id': 3, 'palavras': [ # Login Único
        'LOGIN','LOGIN UNICO','CADASTRO UNICO','SENHA PROVISORIA'
    ]},
    {'id': 5, 'palavras': [ # PGMEI
        'ICMS','IPI','ISS','GUIA DAS','gerar DAS','imprimir DAS'
    ]},
    {'id': 4, 'palavras': [ # IRPF 2020
        'DIRPF','IRPF','RFB','SRF','GCAP','ECAC','CAC','DARF','MAED','IMPOSTO','RENDA','RERCT','PESSOA FISICA','RECEITA FEDERAL','DEPENDENTE','DEPENDENTES',
        'restituir','declarar','isento','isencao','isencoes','deduzir','deducao','deducoes','dedutivel','abater',
        'entregar DECLARACAO','entregar IRPF','entregar IMPOSTO','dispensa entregar',
        'PRAZO estendido','PRAZO prorrogar','PRAZO adiar','PRAZO entregar','ATRASO entregar','PRAZO OUTRO PAIS','PRAZO FORA PAIS',
        'CARNE LEAO','GANHO CAPITAL','valor BENS','valor BEM','relacionar BENS','SAIDA DEFINITIVA','PROGRAMA GERADOR IR','IMPOSTO FONTE','MALHA FINA','atividade RURAL',
        'rendimentos recebidos ACUMULADAMENTE','NUMERO RECIBO','RECIBO ENTREGA','patrimonio','pagamento operadora','fonte pagadora',
        'valor doacao','valor doacoes','limite doacao','limite doacoes','despesa instrucao','despesa educacao','despesa medica','despesa saude',
        'desconto empregada','desconto diarista','rendimento','valor propriedade','retificar valor','retificar declaracao','valor recebido'
    ]},
    {'id': 5, 'palavras': [ # PGMEI
        'PGMEI','MEI','microempreendedor','SISMEI','DASN','CNAE','NOME FANTASIA','EMPREENDEDOR INDIVIDUAL'
    ]},
    {'id': 2, 'palavras': [ # COVID-19
        'COVID','CORONA','CORONAVIRUS','PANDEMIA','comorbidade',
        'distancia contaminar','distancia contagio','distancia SOCIAL','ALCOOL GEL'
    ]},
    {'id': 8, 'palavras': [ # Tuberculose
        'tuberculose','BACILO','KOCH','bacteria pulmonar'
    ]},
    {'id': 2, 'palavras': [ # COVID-19
        'risco','china','MASCARA','lavar embalagem'
    ]},
    {'id': 0, 'palavras': [ # Assuntos não-relacionados: INSS, AIDS, Dengue, Lombriga, Hipertensão, Ansiedade
        'INSS','aposentar','grupo TRANSICAO','regra TRANSICAO',
        'HIV','AIDS','ATO SEXUAL','IMUNODEFICIENCIA',
        'DENGUE','MOSQUITO',
        'lombriga','verme',
        'PRESSAO ALTA','hipertenso','PRESSAO ARTERIAL',
        'ANSIEDADE'
    ]},
    {'id': 2, 'palavras': [ # COVID-19
        'virus','contaminar','contagio','infectar'
    ]},
    {'id': 3, 'palavras': [ # Login Único
        'SENHA','LOGAR','TERMO USO','GOV BR','PORTAL GOVERNO','PORTAL servico',
        'DADOS apagar','DADOS remover','DADOS bloquear','DADOS desautorizar','DADOS desativar','DADOS utilizacao','DADOS PESSOAIS','autorizar DADOS'
    ]},
    {'id': 6, 'palavras': [ # Selo Turismo Responsável
        'SELO','SELOS','hospedagem','resort','pousada','hotel','HOTEIS','albergue',
        'turismo protocolo','viagem protocolo','empreendimento protocolo','servico protocolo','ANVISA protocolo','MTUR protocolo','DESTINO SEGURO',
        'higiene','sanitarias',
        'CGU','CONTROLADORIA GERAL UNIAO'
    ]},
    {'id': 1, 'palavras': [ # Alistamento Militar
        'ARRIMO','SOLDO','dispensa'
    ]},
    {'id': 7, 'palavras': [ # Cadastur
        'MARCA promocao','MARCA promover','MARCA divulgar','MARCA evento','USO MARCA','USAR MARCA'
    ]},
    
    {'id':-1, 'palavras': []}, # Representa registro sem classificação
]

### Semente Aleatória

Semente aleatória a ser usada ao longo desse notebook.

Procure manter sempre a mesma semente aleatória (ou sementes aleatórias, caso utilize mais de uma). Deste modo, poderá comparar a evolução entre diferentes técnicas e também obter a reprodutibilidade exigida pelo regulamento do desafio.

O valor da semente abaixo é apenas ilustrativo, fique à vontade para alterá-lo.

In [ ]:
random_state=112020

### Elementos de NLP

In [ ]:
# Tokenizador: utilizado para separar uma frase em palavras.
tokenizer = RegexpTokenizer(r'\w+')

# stopwords do português
stopwords = nltk.corpus.stopwords.words('portuguese')

# retira keywords da lista de stopwords
for item in keywords:
    for phrase in item['palavras']:
        for keyword in phrase.split():
            kword = keyword.strip().lower()
            if kword in stopwords:
                stopwords.remove(kword)


### Nomes dos Arquivos Utilizados

Nomes dos arquivos que serão utilizados ao longo deste notebook.

In [ ]:
# Caminho dos arquivos de entrada
input_path = '../input/desafio-ia-2020-pln-chatbots/'

# Caminho dos arquivos de saída
output_path = '../working/'

# Nome do arquivo CSV onde estão armazenadas as perguntas rotuladas, para treino e teste.
arquivo_treino_testes = input_path + 'treino_testes.csv'

# Nome do arquivo CSV onde serão armazenadas as perguntas não rotuladas, para classificação e submissão.
# Cada pergunta aqui conterá um identificador que deverá ser mantido.
arquivo_submissao = input_path + 'submissao.csv'

# Nome do arquivo que será criado com os rótulos gerados pelo classificador.
# Este é o arquivo que será submetido à página do desafio e que gerará um score.
# Ele deverá conter apenas os identificadores das perguntas e os identificadores dos respectivos bots.
arquivo_submissao_classificado = output_path + 'submissao_equipe_{}.csv'

# Nome do arquivo CSV de dados de submissão processados, para depuração
arquivo_dados_processados = output_path + 'processado_{}.csv'

# Nome do arquivo CSV de dados classificados por keywords, para depuração
arquivo_dados_classificados = output_path + 'classificado_{}.csv'

# Nome do arquivo CSV de dados não-classificados por keywords, para depuração
arquivo_dados_nao_classificados = output_path + 'nao_classificado_{}.csv'


In [ ]:
# Apaga arquivos CSV de sessões anteriores no diretório de saída
import os
from os import walk

for (dirpath, dirnames, filenames) in walk(output_path):
    for filename in filenames:
        if filename.endswith('.csv'):
            os.remove(output_path + filename)
            print(f'delete: {filename} ok.')

# Carregar os dados de treino e teste

Carrega os dados do arquivo CSV com as perguntas rotuladas com os *ids* dos bots aos quais pertencem. Os rótulos variam de **1** a **8**, como pode ser conferido na lista de bots definida na seção **inicialização**. Estas perguntas serão usadas para treinamento e teste do(s) classificador(es).

Todas as perguntas deste arquivo estão relacionadas a um (e apenas um) dos bots listados acima. Ou seja, não há nenhuma pergunta rotulada como **0** (zero). Este tipo de pergunta (não relacionada a nenhum dos bots) aparecerá **apenas no arquivo de submissão**.

### Dados de Treino e Teste

In [ ]:
# Carrega o arquivo CSV
df = pd.read_csv(arquivo_treino_testes, index_col=None, engine='python', sep =',', encoding="utf-8")
print('Total de registros carregados:',len(df))

# Exibe uma amostra dos dados carregados
df.tail(-1)

In [ ]:
# Distribuição das classes nos dados fornecidos. Note que não há nenhum pergunta rotulada como "0".
df.bot_id.value_counts()

### Dados de Submissão

In [ ]:
# Carrega o arquivo CSV
df_subm = pd.read_csv(arquivo_submissao, index_col=None, engine='python', sep =',', encoding="utf-8")
print('Total de registros carregados:',len(df))

# Exibe uma amostra dos dados carregados
df_subm.tail(-1)

# Preparar os textos para classificação

A preparação dos dados é uma das etapas mais importantes para se obter uma boa performance na classificação de textos, e pode significar a diferença entre o sucesso e o fracasso de um projeto.

### Dados de treino/teste

In [ ]:
# Limpa os dados, preparando-os para classificação.
df['pergunta_original'] = df['pergunta']
df['pergunta'] = df['pergunta'].apply(limpar_texto)
df.tail(-1)

### Dados para submissão

In [ ]:
# Limpa os dados, preparando-os para classificação.
df_subm['pergunta_original'] = df_subm['pergunta']
df_subm['pergunta'] = df_subm['pergunta'].apply(limpar_texto)
df_subm.tail(-1)

# Obter um vetorizador
Nessa etapa, vamos obter um *vetorizador*. Seu objetivo é converter os textos em **vetores numéricos**, para então submetê-los aos algoritmos de classificação.

In [ ]:
tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer.fit_transform(df['pergunta'].tolist())

# Escolher um modelo preditivo, treinar e testar o modelo


In [ ]:
# Combinações de classificador/vetorizador
# Vetorizador TfidfVectorizer
estimators = [
  {'est': LinearSVC(), 
   'grid':{
       'random_state':[random_state],
       'C':[1],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
  {'est': SVC(), 
   'grid':{
       'random_state':[random_state],
       'C':[10],
       'gamma':[1],
       'kernel':['linear'],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
  {'est': SGDClassifier(), 
   'grid':{
       'random_state':[random_state],
       'max_iter':[10000],
       'loss': ['modified_huber'],
       'class_weight':[None],
       'tol':[1e-3],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
#  {'est': ComplementNB(), 
#   'grid':{
#       'norm':[True],
#       'alpha':[1.3],
#   },
#   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
#  {'est': LogisticRegression(), 
#   'grid':{
#       'random_state':[random_state],
#       'max_iter':[10000],
#   },
#   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
#  {'est': XGBClassifier(), 
#   'grid':{ 
#      'random_state': [random_state],
#      'nthread':[4],
#      'objective':['reg:squarederror'],
#      'learning_rate': [.03],
#      'max_depth': [8],
#      'min_child_weight': [4],
#      'subsample': [.7],
#      'colsample_bytree': [.7],
#      'n_estimators': [80],
#   },
#   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
#  {'est': CatBoostClassifier(), 
#   'grid':{ 
#      'random_state':[random_state],
#      'iterations':[100],
#      'silent':[False],
#      'learning_rate': [.03],
#   },
#   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
#  {'est': NearestCentroid(), 
#   'grid':{ 
#      'metric': ['euclidean'], 
#      'shrink_threshold': [None],
#   },
#   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
#  {'est': DecisionTreeClassifier(), 
#   'grid':{ 
#      'random_state':[random_state],
#   },
#   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0, 'params':{}},
]

In [ ]:
%%time
# Treina/testa classificador/vetorizador
n_splits = 5
for estimator in estimators:
    estimator['est'], estimator['est_name'], estimator['vec_name'], estimator['accurracy'], estimator['precision'], estimator['recall'], estimator['f1'], estimator['params'] = build(df['pergunta'],df['bot_id'],estimator['vec'],estimator['est'],estimator['grid'],n_splits)


In [ ]:
# Seleção do melhor classificador/vetorizador
def get_f1(estimator):
    return estimator.get('f1')

estimators.sort(key=get_f1, reverse=True)

for estimator in estimators:
    e_name, v_name, f1, acc, prec, rc, e_params = estimator['est_name'], estimator['vec_name'], round(estimator['f1'],4), round(estimator['accurracy'],4), round(estimator['precision'],4), round(estimator['recall'],4), estimator['params']
    print(f"{e_name} - {v_name} - F1: {f1}, Acc: {acc}, Prec: {prec}, Rec: {rc} - Params: {e_params}")

# escolhe melhor classificador/vetorizador
clf = estimators[0]['est']
vectorizer = estimators[0]['vec']
print('\nSelecionado: ',estimators[0]['est_name'],'-',estimators[0]['vec_name'])

In [ ]:
# escolhe manualmente classificador/vetorizador
#index = 1
#clf = estimators[index]['est']
#vectorizer = estimators[index]['vec']
#print('\nSelecionado: ',estimators[index]['est_name'],'-',estimators[index]['vec_name'])

# Classificar os registros não rotulados para o desafio

De forma cíclica, repita os passos acima testando outras preparações de dados, outros vetorizadores, outros parâmetros e outras técnicas de classificação. Quando estiver satisfeito com a performance do seu modelo, siga os passos abaixo.

Vamos agora treinar o classificador com **todos** os registros pré-rotulados. Este classificador será então utilizado para inferir os bots das perguntas não rotuladas do desafio, como veremos a seguir.

### Classificação por palavras-chave

Este tipo de classificação visa melhorar a performance do estimador quando uma frase possui uma palavra-chave que define já classe automaticamente.

In [ ]:
%%time
# Treina o classificador KeywordClassifier
dummyVectorizer = DummyVectorizer()
kwClassifier = KeywordClassifier()
df_keywords = pd.DataFrame(keywords,columns=['id','palavras'])

X_train = dummyVectorizer.transform(df_keywords['palavras'].tolist())
y_train = df_keywords['id']
kwClassifier.fit(X_train, y_train)

In [ ]:
%%time
# Testa performance do classificador/vetorizador
n_splits = 5
e, e_name, v_name, acc, prec, rc, f1, e_params = build(df['pergunta'],df['bot_id'],dummyVectorizer,kwClassifier,None,n_splits,False)

print(f"{e_name} - {v_name} - F1: {round(f1,4)}, Acc: {round(acc,4)}, Prec: {round(prec,4)}, Rec: {round(rc,4)} - Params: {e_params}")

In [ ]:
%%time
# Classifica registros se encontrar palavras-chave correspondentes
X_test = dummyVectorizer.transform(df_subm['pergunta'].tolist())
y_predicted = kwClassifier.predict(X_test)
df_subm['bot_id'] = y_predicted

# Reordena registros
df_subm.sort_values('bot_id',inplace=True)
df_subm.reset_index(inplace=True,drop=True)

# Nomeia registros para facilitar depuracao
df_subm['bot_nome'] = ''
for i in df_subm.index:
    if df_subm['bot_id'][i] != -1:
        df_subm['bot_nome'][i] = bots[df_subm['bot_id'][i]]['nome']
# Exibe registros
df_subm.head(-1)

In [ ]:
# salva registros dos dados classificados, para análise
df_test = df_subm[df_subm['bot_id'] != -1]
df_test.to_csv(arquivo_dados_classificados.format((datetime.now() - timedelta(hours=3)).strftime('%Y-%m-%d_%H-%M-%S')), index=False, encoding="utf-8", columns=['id','pergunta_original','bot_id','bot_nome'])

# salva registros dos dados não-classificados, para análise
df_test = df_subm[df_subm['bot_id'] == -1]
df_test.to_csv(arquivo_dados_nao_classificados.format((datetime.now() - timedelta(hours=3)).strftime('%Y-%m-%d_%H-%M-%S')), index=False, encoding="utf-8", columns=['id','pergunta_original','pergunta'])

In [ ]:
%%time
# Treina o classificador com toda a base fornecida.
X_train = vectorizer.transform(df['pergunta'].tolist()).toarray()
y_train =  df['bot_id']
clf.fit(X_train, y_train)

In [ ]:
# Vetoriza os textos que serão classificados.
# Somente os que já não foram classificados anteriormente
df_test = df_subm[df_subm['bot_id'] == -1]
X_test = vectorizer.transform(df_test['pergunta'].tolist()).toarray()

In [ ]:
# Executa a classificação dos registros não rotulados.
# Somente os que já não foram classificados anteriormente
y_predicted = clf.predict(X_test)
df_test['bot_id'] = y_predicted
for i in df_test.index:
    df_subm['bot_id'][i] = df_test['bot_id'][i]

# Nomeia registros para facilitar depuracao
df_subm['bot_nome'] = ''
for i in df_subm.index:
    df_subm['bot_nome'][i] = bots[df_subm['bot_id'][i]]['nome']
# Exibe registros
df_subm.head(-1)

### Gera matriz de probabilidades

A matriz de probabilidades permite avaliar o quanto o estimador acertou a classificação de cada amostra, e assim, invalidar o resultado para classificações de baixa probabilidade (provavelmente deveriam ser da classe 0, 'Nenhum')

In [ ]:
# Gera matriz de probabilidades
y_proba = []
try:
    y_proba = clf.predict_proba(X_test)
except:
    try:
        y_proba = clf.decision_function(X_test)
    except:
        pass

df_proba = pd.DataFrame()   

if len(y_proba) > 0:
    df_proba = pd.DataFrame(y_proba,columns=[1,2,3,4,5,6,7,8])
    # normaliza para 0..1
    pmin = df_proba.min().min()
    pmax = df_proba.max().max()
    df_proba = (df_proba - pmin) / (pmax - pmin)
    probas = []
    for i in df_proba.index:
        # pega a distância entre min e max
        n = [df_proba.iloc[i].min(), df_proba.iloc[i].max()]
        proba = abs(n[1] - n[0])
        probas.append(proba)
    df_proba = pd.concat([df_proba,pd.DataFrame(probas,columns=['proba'])],axis=1)
    
# Exibe matriz
df_proba.head(-1)

In [ ]:
# Corrige classificacao de baixa probabilidade
# Provavelmente, esses registros pertencem a classe Nenhum (0)

# considerei um valor de probabilidade mínimo aceitável
proba_min = 0.2
df_subm['proba'] = df_proba['proba']
bid_zero = bots[0]['id'] # classe zero
count = 0
for i in df_proba.index:
    proba = df_proba['proba'][i]
    if proba < proba_min: 
        df_subm['bot_id'][i] = bid_zero
        count = count + 1

print(f'Desclassificados {count}/{len(df_subm.index)} registros com probabilidade abaixo de {proba_min*100.0}%.')
df_subm['proba'] = df_subm['proba'].fillna(1.0) # preenche NaN com 1.0

In [ ]:
# Nomeia registros para facilitar depuracao
df_subm['bot_nome'] = ''
for i in df_subm.index:
    df_subm['bot_nome'][i] = bots[df_subm['bot_id'][i]]['nome']
# Exibe registros
df_subm.head(-1)

In [ ]:
# salva registros dos dados processados, para análise
df_subm.to_csv(arquivo_dados_processados.format((datetime.now() - timedelta(hours=3)).strftime('%Y-%m-%d_%H-%M-%S')), index=False, encoding="utf-8")

# Salvar os registros classificados.

Apenas os identificadores das perguntas e os identificadores dos respectivos bots devem armazenados!

In [ ]:
# salva registros classificados
df_subm.to_csv(arquivo_submissao_classificado.format((datetime.now() - timedelta(hours=3)).strftime('%Y-%m-%d_%H-%M-%S')), index=False, encoding="utf-8", columns=['id','bot_id'])

# Submeter os resultados à página do desafio

Entre na página do desafio e faça o upload do arquivo *csv* obtido acima, com as classificações realizadas pelo seu modelo. Esse arquivo deve conter apenas as colunas "*id*" e "*id_bot*". O site irá calcular automaticamente sua métrica de acerto.

Se o score obtido estiver nas três primeiras posições, faça um versionamento do código. Se esta posição se mantiver até o final da competição, ele será auditado para verificação de reprodutibilidade.

Você pode fazer quantas tentativas desejar, até atingir um limite diário (consulte o regulamento). Use isso para melhorar suas métricas.